In [162]:
import pandas as pd
import numpy as np
import re

In [163]:
cdata = pd.read_csv("Data/Football-Scenarios-DFE-832307.csv")

In [164]:
cdata.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,antecedent,antecedent:confidence,orig_antecedent,antecedent_gold,option1,option2,option3,option4,option5
0,831005673,False,finalized,5,11/20/15 20:20,kick a field goal,0.8092,It is first down and 10. The ball is on your o...,NaN,punt,kick a field goal,run,pass,kneel down
1,831005674,False,finalized,5,11/18/15 21:59,kick a field goal,1.0000,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down
2,831005675,False,finalized,5,11/20/15 22:43,kick a field goal,0.6211,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down
3,831005676,False,finalized,5,11/19/15 7:41,kick a field goal,0.8073,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down
4,831005677,False,finalized,5,11/21/15 8:01,kick a field goal,1.0000,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down


In [165]:
set(cdata.option1)

{' punt'}

In [166]:
set(cdata.option2)

{' kick a field goal'}

In [167]:
set(cdata.option3)

{' run'}

In [168]:
set(cdata.option4)

{' pass'}

In [169]:
set(cdata.option5)

{' kneel down'}

In [170]:
len(cdata)

3730

In [171]:
cdata.orig_antecedent.str.split('.').head()


0    [It is first down and 10,  The ball is on your...
1    [It is second down and inches,  The ball is on...
2    [It is second down and inches,  The ball is on...
3    [It is second down and inches,  The ball is on...
4    [It is second down and inches,  The ball is on...
dtype: object

In [172]:
split_scenarios = cdata.orig_antecedent.str.split('.').tolist()
split_scenarios = [scenario[0:-1] for scenario in split_scenarios]

In [173]:
#down = []
#ytg = []
#fieldpos = []
#quarter = []
#timeonclock = []
#scorediff = []
#for scenario in split_scenarios:
#    data = [np.NAN,np.NAN,np.NAN,np.NAN,np.NAN,np.NAN]
#    for line in scenario:
#        split_line = line.strip().split(' ')
#        if 'It is' in line:
#            data[0] = split_line[2]
#            if len(split_line) == 6:
#                data[1] = split_line[5]
#        if 'The ball' in line:
#            if len(split_line) == 7:
#                data[2] = int(split_line[5]) * -1 
#            else:
#                data[2] = int(split_line[6])
#        if 'There is' in line:
#            data[3] = '_'
#            
#        print(split_line)
#        print(data)
#        print()         

 **down | yards to 1st down | field position | quarter | time left on the clock | score differential**

Avoid declaring empty lists

Avoid making dummy tuples

Vectorize!

Make code stateless... single operation / multiple data manipulations are better for processors to ... well ... process!

In [174]:
def convert_ordinal(value):
    if value == 'first':
        return 1
    elif value == 'second':
        return 2
    elif value == 'third':
        return 3
    elif value == 'fourth':
        return 4
    else:
        return value
    

In [175]:
def convert_clock(value):
    m = re.search('(.+?) minute', value)
    s = re.search('(.+?) second', value)
    if m:
        return int(m.group(1)) * 60
    elif s:
        return int(s.group(1))
    else:
        return value

In [176]:
def convert_scoredelta(value):
    d = re.search('down by (.+?)', value)
    u = re.search('up by (.+?)', value)
    
    if d:
        return int(d.group(1)) * -1
    elif u:
        return int(u.group(1))
    else:
        return value

In [ ]:
def convert_fieldpos(value):
    

In [187]:
def extract(line):
    #line = ''.join(scenario)
    
    down = re.search('It is (.+?) down', line)
    ytg = re.search('down and (.+?). ',line)
    fieldpos = re.search("your (.+?) yardline", line)
    if fieldpos == None:
        fieldpos = re.search("(your .+?) yardline", line)
    quarter = re.search('the (.+?) quarter', line)
    clock = re.search('There is (.+?) left', line)
    scoredelta = re.search('You are (.+?) points',line)
    
    extraction = (
        line,
        convert_ordinal(down.group(1)) if down else np.NaN,
        int(ytg.group(1).replace('inches','0')) if ytg else np.NaN,
        #(fieldpos.group(1).replace('your ', '-') if len(fieldpos.group(1)) < 8 else fieldpos.group(1).replace("opponent's ",'')) if fieldpos else np.NaN,
        fieldpos.group(1) if fieldpos else np.NaN,
        convert_ordinal(quarter.group(1)) if quarter else np.NaN,
        convert_clock(clock.group(1)) if clock else np.NaN,
        convert_scoredelta(scoredelta.group(1)) if scoredelta else np.NaN
    )
    return extraction

In [188]:
data = [
    extract(scenario)
    for scenario in cdata.orig_antecedent
]

In [189]:
df = pd.DataFrame.from_records(data, columns=['scenario','down','ytg','fieldpos','quarter','clock','scoredelta'])
df

,scenario,down,ytg,fieldpos,quarter,clock,scoredelta
0,It is first down and 10. The ball is on your o...,1,10,opponent's 20,2,3,-3
1,It is second down and inches. The ball is on y...,2,0,opponent's 5,2,3,-3
2,It is second down and inches. The ball is on y...,2,0,opponent's 20,2,3,-3
3,It is second down and inches. The ball is on y...,2,0,opponent's 5,4,3,-3
4,It is second down and inches. The ball is on y...,2,0,opponent's 20,4,3,-3
5,It is second down and inches. The ball is on y...,2,0,45,4,3,-7
6,It is second down and inches. The ball is on y...,2,0,20,4,3,-7
7,It is second down and 3. The ball is on your o...,2,3,opponent's 5,4,3,-3
8,It is second down and 3. The ball is on your o...,2,3,opponent's 20,4,3,-3
9,It is second down and 3. The ball is on your o...,2,3,opponent's 20,2,3,-7


In [180]:
cdata[['scenario','down','ytg','fieldpos','quarter','clock','scoredelta']]=df

In [181]:
cdata

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,antecedent,antecedent:confidence,orig_antecedent,antecedent_gold,option1,...,option3,option4,option5,scenario,down,ytg,fieldpos,quarter,clock,scoredelta
0,831005673,False,finalized,5,11/20/15 20:20,kick a field goal,0.8092,It is first down and 10. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your o...,1,10,20,2,3,-3
1,831005674,False,finalized,5,11/18/15 21:59,kick a field goal,1.0000,It is second down and inches. The ball is on y...,NaN,punt,...,run,pass,kneel down,It is second down and inches. The ball is on y...,2,0,5,2,3,-3
2,831005675,False,finalized,5,11/20/15 22:43,kick a field goal,0.6211,It is second down and inches. The ball is on y...,NaN,punt,...,run,pass,kneel down,It is second down and inches. The ball is on y...,2,0,20,2,3,-3
3,831005676,False,finalized,5,11/19/15 7:41,kick a field goal,0.8073,It is second down and inches. The ball is on y...,NaN,punt,...,run,pass,kneel down,It is second down and inches. The ball is on y...,2,0,5,4,3,-3
4,831005677,False,finalized,5,11/21/15 8:01,kick a field goal,1.0000,It is second down and inches. The ball is on y...,NaN,punt,...,run,pass,kneel down,It is second down and inches. The ball is on y...,2,0,20,4,3,-3
5,831005678,False,finalized,5,11/21/15 18:25,pass,1.0000,It is second down and inches. The ball is on y...,NaN,punt,...,run,pass,kneel down,It is second down and inches. The ball is on y...,2,0,-45,4,3,-7
6,831005679,False,finalized,5,11/20/15 20:31,pass,1.0000,It is second down and inches. The ball is on y...,NaN,punt,...,run,pass,kneel down,It is second down and inches. The ball is on y...,2,0,-20,4,3,-7
7,831005680,False,finalized,5,11/20/15 19:10,kick a field goal,0.7953,It is second down and 3. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is second down and 3. The ball is on your o...,2,3,5,4,3,-3
8,831005681,False,finalized,5,11/20/15 18:20,kick a field goal,1.0000,It is second down and 3. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is second down and 3. The ball is on your o...,2,3,20,4,3,-3
9,831005682,False,finalized,5,11/20/15 9:58,pass,0.5874,It is second down and 3. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is second down and 3. The ball is on your o...,2,3,20,2,3,-7


In [183]:
cdata[(cdata.clock <= 10)&(cdata.down == 1)&(cdata.fieldpos )]

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,antecedent,antecedent:confidence,orig_antecedent,antecedent_gold,option1,...,option3,option4,option5,scenario,down,ytg,fieldpos,quarter,clock,scoredelta
0,831005673,False,finalized,5,11/20/15 20:20,kick a field goal,0.8092,It is first down and 10. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your o...,1,10,20,2,3,-3
24,831005697,False,finalized,5,11/19/15 17:44,kick a field goal,0.5694,It is first down and 10. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your o...,1,10,40,2,3,-3
25,831005698,False,finalized,5,11/19/15 17:05,pass,0.8155,It is first down and 10. The ball is on your 4...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your 4...,1,10,-45,2,3,-3
26,831005699,False,finalized,5,11/19/15 2:55,pass,0.3762,It is first down and 10. The ball is on your 2...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your 2...,1,10,-20,2,3,-3
39,831005712,False,finalized,5,11/20/15 20:00,kick a field goal,1.0000,It is first down and 10. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your o...,1,10,20,4,3,-3
40,831005713,False,finalized,5,11/20/15 21:31,pass,0.6238,It is first down and 10. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your o...,1,10,40,4,3,-3
41,831005714,False,finalized,5,11/21/15 16:34,pass,0.8247,It is first down and 10. The ball is on your 4...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your 4...,1,10,-45,4,3,-3
42,831005715,False,finalized,5,11/20/15 16:01,pass,0.8031,It is first down and 10. The ball is on your 2...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your 2...,1,10,-20,4,3,-3
56,831005729,False,finalized,5,11/18/15 22:07,kick a field goal,1.0000,It is first down and 10. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your o...,1,10,20,2,3,-7
57,831005730,False,finalized,5,11/20/15 19:04,kick a field goal,0.5984,It is first down and 10. The ball is on your o...,NaN,punt,...,run,pass,kneel down,It is first down and 10. The ball is on your o...,1,10,40,2,3,-7
